<a href="https://colab.research.google.com/github/Pearl5007/AI-CAPSTONE-PROJECT/blob/main/ai_website_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# =====================================================
# FINAL WORKING VERSION — FLASK + KNN + GITHUB DATA + UI
# =====================================================

!pip install -q flask scikit-learn pandas numpy

import pandas as pd
import numpy as np
from flask import Flask, request, render_template_string
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from google.colab import output
import threading, time


# =====================================================
# LOAD DATASET FROM GITHUB
# =====================================================
url = "https://raw.githubusercontent.com/Pearl5007/AI-CAPSTONE-PROJECT/refs/heads/main/data/wisc_bc_data.csv"
df = pd.read_csv(url)
print("Dataset loaded successfully!")

# Drop ID column if exists
if "id" in df.columns:
    df = df.drop(columns=["id"])

# Fix column name errors if they exist
rename_map = {
    "points_mean": "concave_points_mean",
    "dimension_mean": "fractal_dimension_mean",
    "points_se": "concave_points_se",
    "dimension_se": "fractal_dimension_se",
    "points_worst": "concave_points_worst",
    "dimension_worst": "fractal_dimension_worst",
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# Detect target column
target_col = None
for c in df.columns:
    if c.lower() in ("diagnosis", "target", "label"):
        target_col = c
        break
if target_col is None:
    target_col = df.columns[-1]  # assume last column


# Clean and map labels
def map_label(v):
    s = str(v).strip().upper()
    if s.startswith("M"):
        return "Malignant"
    if s.startswith("B"):
        return "Benign"
    try:
        return "Malignant" if float(s) > 0.5 else "Benign"
    except:
        return None

y = df[target_col].astype(str).map(map_label)
valid = y.isin(["Benign", "Malignant"])
df = df.loc[valid]
y = y.loc[valid]

# Numeric features only
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != target_col]

FEATURES = numeric_cols
X = df[FEATURES].astype(float)
y = y.map(lambda v: 0 if v == "Benign" else 1)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)

# Train KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

print("Train accuracy:", knn.score(X_train, y_train))
print("Test accuracy:", knn.score(X_test, y_test))


# =====================================================
# FULL HTML TEMPLATE (YOUR EXACT CODE)
# =====================================================

HTML = r"""
<!doctype html>
<html>
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<title>BREAST CANCER DETECTION</title>
<style>
:root{
  --bg1: #f6fbff;
  --bg2: #f2fff9;
  --card: rgba(255,255,255,0.78);
  --glass: 12px;
  --accent1: #00c6ff;
  --accent2: #9be7ff;
  --accent3: #c8e9ff;
  --muted: #626f7a;
}

/* Background & blobs */
html,body{height:100%;margin:0;font-family:Inter, 'Segoe UI', Roboto, Arial;color:#082033;
 background: radial-gradient(800px 400px at 10% 10%, rgba(156,136,255,0.06), transparent 10%),
             radial-gradient(700px 350px at 90% 90%, rgba(0,198,255,0.06), transparent 10%),
             linear-gradient(180deg,var(--bg1),var(--bg2));
 -webkit-font-smoothing:antialiased; -moz-osx-font-smoothing:grayscale;
}

/* floating blobs */
.blob{ position:fixed; border-radius:50%; filter: blur(36px); opacity:0.6; pointer-events:none; mix-blend-mode:screen; transform: translateZ(0); }
.blob.a{ width:380px; height:380px; left:-100px; top:-80px; background: linear-gradient(45deg,var(--accent3),var(--accent1)); animation: floatA 16s ease-in-out infinite; }
.blob.b{ width:420px; height:420px; right:-140px; bottom:-120px; background: linear-gradient(45deg,var(--accent2),var(--accent3)); animation: floatB 18s ease-in-out infinite; }
@keyframes floatA { 0%{transform:translateY(0)}50%{transform:translateY(-24px)}100%{transform:translateY(0)} }
@keyframes floatB { 0%{transform:translateY(0)}50%{transform:translateY(20px)}100%{transform:translateY(0)} }

/* main layout */
.wrap{ max-width:1100px; margin:34px auto; padding:18px; position:relative; z-index:2; }
.header{ display:flex; align-items:center; justify-content:space-between; gap:12px; margin-bottom:18px; }
.title{ font-weight:800; letter-spacing:1px; font-size:20px; color:#042b3a; }
.header-sub{ color:var(--muted); font-size:0.95rem; }

/* grid */
.grid{ display:grid; grid-template-columns: 1fr 380px; gap:18px; align-items:start; }

/* card glass */
.card{ background:var(--card); border-radius:16px; padding:20px; box-shadow: 0 12px 40px rgba(6,22,34,0.06); backdrop-filter: blur(var(--glass)); border: 1px solid rgba(255,255,255,0.6); transition: transform .28s ease, box-shadow .28s ease;}
.card:hover{ transform: translateY(-6px); box-shadow:0 22px 50px rgba(6,22,34,0.09); }

/* form */
.form-title{ font-size:1.05rem; font-weight:700; margin-bottom:8px; color:#042b3a;}
.inputs{ display:grid; grid-template-columns: repeat(2,1fr); gap:12px; }
label{ display:block; font-weight:600; font-size:0.92rem; color:#0b3b4a; margin-bottom:6px;}
input[type=number]{ width:100%; padding:12px 12px; border-radius:12px; border:1px solid rgba(8,20,30,0.06); font-size:0.95rem; background: rgba(255,255,255,0.9); transition: box-shadow .16s, transform .12s; }
input[type=number]:focus{ outline:none; box-shadow:0 12px 28px rgba(0,198,255,0.12); transform: translateY(-2px); border-color: rgba(0,198,255,0.28); }

/* CTA */
.cta{ margin-top:12px; display:flex; gap:10px; align-items:center; }
.btn{ padding:12px 16px; border-radius:12px; border:none; cursor:pointer; font-weight:800; color:#042b3a;
      background: linear-gradient(90deg,var(--accent1),var(--accent2)); box-shadow: 0 10px 30px rgba(0,198,255,0.12); transition: transform .14s ease, box-shadow .14s ease; position:relative; overflow:hidden;}
.btn:hover{ transform: translateY(-4px); box-shadow:0 20px 48px rgba(0,198,255,0.16); }

/* ripple */
.ripple{ position:absolute; border-radius:999px; transform:scale(0); background: rgba(255,255,255,0.5); animation: ripple 0.65s linear; }
@keyframes ripple{ to{ transform:scale(3); opacity:0; } }

/* result */
.result{ margin-top:14px; padding:14px; border-radius:12px; font-weight:800; text-align:center; }
.result.good{ background: rgba(34,197,94,0.08); color:#065f46; box-shadow: 0 8px 30px rgba(34,197,94,0.06); }
.result.bad{ background: rgba(239,68,68,0.08); color:#7f1d1d; box-shadow: 0 8px 30px rgba(239,68,68,0.06); }

/* right panel */
.panel .meta{ color:var(--muted); font-size:0.95rem; margin-bottom:8px; }
.hint{ padding:12px; border-radius:10px; background: rgba(255,255,255,0.9); box-shadow: 0 8px 28px rgba(6,22,34,0.04); color:#05323a; }

/* spinner overlay */
.overlay{ display:none; position:fixed; inset:0; align-items:center; justify-content:center; z-index:99; background: rgba(3,6,8,0.28); }
.spinner{ width:72px; height:72px; border-radius:50%; border:10px solid rgba(255,255,255,0.09); border-top-color: #00d4ff; animation:spin 1s linear infinite; }
@keyframes spin{ to{ transform:rotate(360deg); } }

/* responsive */
@media (max-width:980px){ .grid{ grid-template-columns: 1fr; } .panel{ order:2; } }
</style>
</head>
<body>
  <div class="blob a"></div><div class="blob b"></div>

  <div class="wrap">
    <div class="header">
      <div>
        <div class="title">BREAST CANCER DETECTION</div>
        <div class="header-sub">Accurate, fast, and designed for clarity</div>
      </div>
      <div style="text-align:right;">
        <div style="font-size:0.92rem; color:var(--muted)">Model test accuracy: <strong>{{ test_acc }}%</strong></div>
      </div>
    </div>

    <div class="grid">
      <div class="card">
        <div class="form-title">Enter patient feature values</div>

        <form id="predictForm" method="POST" action="/predict">
          <div class="inputs">
            {% for f in features %}
            <div>
              <label>{{ f.replace('_',' ').title() }}</label>
              <input type="number" step="any" name="{{ f }}" required>
            </div>
            {% endfor %}
          </div>

          <div class="cta">
            <button class="btn" id="predictBtn" type="submit">Predict</button>
          </div>

          <div id="resultHolder">
            {% if result is defined and result is not none %}
              <div class="result {{ 'good' if result=='Benign' else 'bad' }}">{{ result }}</div>
            {% endif %}
          </div>
        </form>

      </div>

      <aside class="card panel">
        <div class="meta">About</div>
        <div class="hint">This demo uses a KNN model trained on your GitHub dataset.</div>

        <div style="margin-top:6px;" class="meta">Features used</div>
        <div class="hint">{{ features | join(', ') }}</div>
      </aside>
    </div>
  </div>

  <!-- spinner -->
  <div class="overlay" id="overlay">
    <div class="spinner"></div>
  </div>

<script>
  // ripple effect for button
  document.addEventListener('click', function(e){
    const t = e.target;
    if(t.classList && t.classList.contains('btn')){
      const circle = document.createElement('span');
      circle.className = 'ripple';
      const rect = t.getBoundingClientRect();
      const size = Math.max(rect.width, rect.height);
      circle.style.width = circle.style.height = size + 'px';
      circle.style.left = (event.clientX - rect.left - size/2) + 'px';
      circle.style.top = (event.clientY - rect.top - size/2) + 'px';
      t.appendChild(circle);
      setTimeout(()=> circle.remove(), 700);
    }
  });

  // show spinner on submit
  const form = document.getElementById('predictForm');
  const overlay = document.getElementById('overlay');
  form.addEventListener('submit', function(){ overlay.style.display = 'flex'; });
</script>
</body>
</html>
"""


# =====================================================
# FLASK APP
# =====================================================

app = Flask(__name__)

@app.route("/", methods=["GET"])
def index():
    test_acc = round(knn.score(X_test, y_test) * 100, 2)
    return render_template_string(HTML, features=FEATURES, test_acc=test_acc)

@app.route("/predict", methods=["POST"])
def predict():
    values = [float(request.form.get(f)) for f in FEATURES]
    arr = np.array(values).reshape(1, -1)
    arr_scaled = scaler.transform(arr)
    pred = knn.predict(arr_scaled)[0]
    result = "Benign" if pred == 0 else "Malignant"

    test_acc = round(knn.score(X_test, y_test) * 100, 2)
    return render_template_string(HTML, features=FEATURES, result=result, test_acc=test_acc)


def run():
    app.run(port=7860, debug=False, use_reloader=False)


threading.Thread(target=run).start()
time.sleep(1)
print("Launching app…")
output.serve_kernel_port_as_window(7860)


Dataset loaded successfully!
Train accuracy: 0.9741784037558685
Test accuracy: 0.972027972027972
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 7860 is in use by another program. Either identify and stop that program, or start the server with a different port.


Launching app…
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>